In [ ]:
!pip install pycuda

**ASSUMPTION: Nrows <= Ncols**

**ITERATIONS ARE NOT POSSIBLE DUE TO THE CACHING MECHANISM OF PYCUDA AND THE TEMPLATE COMPILING NEEDS**

In [ ]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

In [ ]:
#numMatrices       = np.uint32(1048576)
#numMatrices       = np.uint32(5)
numMatrices       = np.uint32(4)
Nrows             = np.uint32(4)
Ncols             = np.uint32(4)
maxIter           = np.uint32(Nrows)  # Bidiagonalization iterations

if (Nrows < Ncols):
  LEN = Nrows
else:
  LEN = Nrows - 1

workType          = np.float64
#workType          = np.float32
if workType       == np.float32:
    dtype_str     = "float"
    sturmTol      = np.float32(1e-7)
    #sturmTol      = np.float32(1e-6)
else:
    dtype_str     = "double"
    sturmTol      = np.float64(1e-13)
    #sturmTol      = np.float64(1e-12)

Rearrange - Original storage of the matrices is columnwise, while output storage is interlaced rowwise.

In [ ]:
def makeRearrangeKernel(dtype_str, numMatrices, Nrows, Ncols):

    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}
    #define Ncols {Ncols}

    extern "C" {{
    __global__ void rearrangeKernel(const T * __restrict__ inputMatrices, T * __restrict__ outputMatrices) {{
        int tid = threadIdx.x + blockDim.x * blockIdx.x;
        if (tid < numMatrices) {{
            #pragma unroll
            for (unsigned int i = 0; i < Nrows; i++)
                #pragma unroll
                for (unsigned int j = 0; j < Ncols; j++)
                    outputMatrices[ tid + j * numMatrices + i * (numMatrices * Ncols) ]
                      = inputMatrices[ tid * (Nrows * Ncols) + j * Nrows + i ];
        }}
    }}
    }}"""

Bidiagonalize - Assumes interlaced storage of the matrices

In [ ]:
def makeBidiagonalizeKernel(dtype_str, numMatrices, Nrows, Ncols):
    """
    Restituisce la stringa CUDA per il kernel di bidiagonalizzazione,
    assumendo che l’array d’ingresso sia già “rearrangiato” in formato
    row-major interlacciato tra batch (output di rearrange).

    - dtype_str: "float" o "double"
    - numMatrices, Nrows, Ncols: costanti a compile-time
    """
    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}
    #define Ncols {Ncols}

    // ===============================================
    // COMPUTE LEFT HOUSEHOLDER VECTORS (device function)
    // ===============================================
    __device__ __forceinline__ void computeLeftHouseholderVectors(
        const T * __restrict__ inputMatrices,
        T * __restrict__ outputLeftHouseholderVectors,
        T &betaq,
        const unsigned int tid,
        const unsigned int iterCounter)
    {{
        T x0, norm2 = static_cast<T>(0), mu, v0;

        #pragma unroll
        for (unsigned int i = 0; i < iterCounter; i++)
            outputLeftHouseholderVectors[i] = static_cast<T>(0);

        // Leggiamo, per i = iterCounter+1 ... Nrows-1, l’elemento
        // (row = i, col = iterCounter, batch = tid) nel layout interlacciato:
        #pragma unroll
        for (unsigned int i = iterCounter + 1; i < Nrows; i++)
        {{
            // offset interlacciato:
            //    blocco di righe:    i * (numMatrices * Ncols)
            //    blocco di colonne:  iterCounter * numMatrices
            //    offset batch:       + tid
            const unsigned int idx = i * (numMatrices * Ncols)
                                   + iterCounter * numMatrices
                                   + tid;
            const T buffer = inputMatrices[idx];
            outputLeftHouseholderVectors[i] = buffer;
            norm2 += buffer * buffer;
        }}

        // x0 = elemento (row = iterCounter, col = iterCounter, batch = tid):
        unsigned int idx_x0 = iterCounter * (numMatrices * Ncols)
                            + iterCounter * numMatrices
                            + tid;
        x0 = inputMatrices[idx_x0];

        if (norm2 == static_cast<T>(0))
        {{
            betaq = static_cast<T>(0);
        }}
        else
        {{
            mu = sqrt(x0 * x0 + norm2);
            if (x0 <= static_cast<T>(0))
                v0 = x0 - mu;
            else
                v0 = -norm2 / (x0 + mu);

            T temp = static_cast<T>(1) / v0;
            betaq = static_cast<T>(2) * ((v0 * v0) / (norm2 + (v0 * v0)));
            outputLeftHouseholderVectors[iterCounter] = static_cast<T>(1);

            #pragma unroll
            for (unsigned int s = iterCounter + 1; s < Nrows; s++)
                outputLeftHouseholderVectors[s] *= temp;
        }}
    }}

    // ================================================
    // COMPUTE RIGHT HOUSEHOLDER VECTORS (device function)
    // ================================================
    __device__ __forceinline__ void computeRightHouseholderVectors(
        const T * __restrict__ inputVectors,
        T * __restrict__ outputRightHouseholderVectors,
        T &betap,
        const unsigned int tid,
        const unsigned int iterCounter)
    {{
        T x0, norm2 = static_cast<T>(0), mu, v0;

        #pragma unroll
        for (unsigned int i = 0; i < iterCounter + 1; i++)
            outputRightHouseholderVectors[i] = static_cast<T>(0);

        #pragma unroll
        for (unsigned int i = iterCounter + 2; i < Ncols; i++)
        {{
            const T buffer = inputVectors[i];
            outputRightHouseholderVectors[i] = buffer;
            norm2 += buffer * buffer;
        }}

        x0 = inputVectors[iterCounter + 1];

        if (norm2 == static_cast<T>(0))
        {{
            betap = static_cast<T>(0);
        }}
        else
        {{
            mu = sqrt(x0 * x0 + norm2);
            if (x0 <= static_cast<T>(0))
                v0 = x0 - mu;
            else
                v0 = -norm2 / (x0 + mu);

            T temp = static_cast<T>(1) / v0;
            betap = static_cast<T>(2) * ((v0 * v0) / (norm2 + v0 * v0));
            outputRightHouseholderVectors[iterCounter + 1] = static_cast<T>(1);

            #pragma unroll
            for (unsigned int s = iterCounter + 2; s < Ncols; s++)
                outputRightHouseholderVectors[s] *= temp;
        }}
    }}

    extern "C" {{

    // ===============================================
    // GLOBAL KERNEL: bidiagonalizeKernel
    // ===============================================
    __global__ void bidiagonalizeKernel(
        T *inputMatrices,           // batch di matrici, interlacciate row-major
        const unsigned int maxIter) // massimo numero di iterazioni
    {{
        int tid = threadIdx.x + blockDim.x * blockIdx.x;
        if (tid >= numMatrices) return;

        // Buffer temporanei per ciascun thread:
        T leftHouseholderVectors[Nrows];
        T rightHouseholderVectors[Ncols];
        T inputVectorsForRight[Ncols];
        T wi[Nrows];
        T xi[Ncols];
        T zi[Ncols];
        T betau = static_cast<T>(0), betav = static_cast<T>(0);

        // Iterazioni:
        for (unsigned int iterCounter = 0; iterCounter < maxIter; iterCounter++)
        {{
            if (iterCounter < Ncols - 2)
            {{
                // 1) Compute left Householder
                computeLeftHouseholderVectors(
                    inputMatrices,
                    leftHouseholderVectors,
                    betau,
                    tid,
                    iterCounter);

                // 2) Build inputVectorsForRight (per riflessioni right)
                #pragma unroll
                for (unsigned int j = 0; j < Ncols; j++)
                {{
                    T sum = static_cast<T>(0);
                    #pragma unroll
                    for (unsigned int i = 0; i < Nrows; i++)
                    {{
                        // leggo elemento (row=i, col=j) della matrice "tid" nel layout interlacciato:
                        unsigned int idx_val = i * (numMatrices * Ncols)
                                             + j * numMatrices
                                             + tid;
                        T val = inputMatrices[idx_val];
                        sum += -betau * leftHouseholderVectors[i] * val;
                    }}
                    // aggiungo l’elemento diagonale (row=iterCounter, col=j) di A^(tid):
                    unsigned int idx_diag = iterCounter * (numMatrices * Ncols)
                                          + j * numMatrices
                                          + tid;
                    T diag = inputMatrices[idx_diag];
                    inputVectorsForRight[j] = sum + diag;
                    xi[j] = -sum;
                }}

                // 3) Compute right Householder
                computeRightHouseholderVectors(
                    inputVectorsForRight,
                    rightHouseholderVectors,
                    betav,
                    tid,
                    iterCounter);

                // 4) Build wi
                #pragma unroll
                for (unsigned int i = 0; i < Nrows; i++)
                {{
                    T sum = static_cast<T>(0);
                    #pragma unroll
                    for (unsigned int j = 0; j < Ncols; j++)
                    {{
                        unsigned int idx_val = i * (numMatrices * Ncols)
                                             + j * numMatrices
                                             + tid;
                        T val = inputMatrices[idx_val];
                        sum += betav * rightHouseholderVectors[j] * val;
                    }}
                    wi[i] = sum;
                }}

                // 5) Build zi
                T dot = static_cast<T>(0);
                #pragma unroll
                for (unsigned int i = 0; i < Ncols; i++)
                    dot += xi[i] * rightHouseholderVectors[i];

                #pragma unroll
                for (unsigned int i = 0; i < Ncols; i++)
                    zi[i] = xi[i] - betav * dot * rightHouseholderVectors[i];

                // 6) Update matrix A^(tid)
                #pragma unroll
                for (unsigned int r = 0; r < Nrows; r++)
                {{
                    #pragma unroll
                    for (unsigned int c = 0; c < Ncols; c++)
                    {{
                        unsigned int idx_old = r * (numMatrices * Ncols)
                                             + c * numMatrices
                                             + tid;
                        T oldVal = inputMatrices[idx_old];
                        T L = -leftHouseholderVectors[r] * zi[c];
                        T R = -wi[r] * rightHouseholderVectors[c];
                        inputMatrices[idx_old] = L + R + oldVal;
                    }}
                }}
            }}
            else
            {{
                // Ultima iterazione: solo left Householder
                computeLeftHouseholderVectors(
                    inputMatrices,
                    leftHouseholderVectors,
                    betau,
                    tid,
                    iterCounter);

                // Build xi
                #pragma unroll
                for (unsigned int j = 0; j < Ncols; j++)
                {{
                    T sum = static_cast<T>(0);
                    #pragma unroll
                    for (unsigned int i = 0; i < Nrows; i++)
                    {{
                        unsigned int idx_val = i * (numMatrices * Ncols)
                                             + j * numMatrices
                                             + tid;
                        T val = inputMatrices[idx_val];
                        sum += leftHouseholderVectors[i] * val;
                    }}
                    xi[j] = sum;
                }}

                // Update matrix A^(tid)
                #pragma unroll
                for (unsigned int r = 0; r < Nrows; r++)
                {{
                    #pragma unroll
                    for (unsigned int c = 0; c < Ncols; c++)
                    {{
                        unsigned int idx_old = r * (numMatrices * Ncols)
                                             + c * numMatrices
                                             + tid;
                        T oldVal = inputMatrices[idx_old];
                        inputMatrices[idx_old] = -betau * xi[c] * leftHouseholderVectors[r] + oldVal;
                    }}
                }}
            }}
        }}
    }}
    }}  // extern "C"
    """

Extract diagonals - Assumes interlaced storage of matrices

In [ ]:
def makeExtractDiagonalsKernel(dtype_str, numMatrices, Nrows, Ncols):
    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}
    #define Ncols {Ncols}

    extern "C" {{

    __global__ void extractDiagonalsKernel(
        const T * __restrict__ inputMatrices,
        T * __restrict__ d,
        T * __restrict__ e)
    {{
        int tid = threadIdx.x + blockDim.x * blockIdx.x;
        if (tid >= numMatrices) return;

        // estraiamo solo Nrows elementi per batch
        #pragma unroll
        for (unsigned int i = 0; i < Nrows - 1; i++) {{
            // offset base: r = i, c = i
            unsigned int base_diag = (i * Ncols + i) * numMatrices + tid;
            d[ tid + i * numMatrices ] = inputMatrices[ base_diag ];

            // offset super-diagonale: r = i, c = i+1
            unsigned int base_sup  = (i * Ncols + (i + 1)) * numMatrices + tid;
            e[ tid + i * numMatrices ] = inputMatrices[ base_sup ];
        }}
        // ultimo elemento diagonale (i = Nrows-1)
        unsigned int i = Nrows - 1;
        unsigned int base_last = (i * Ncols + i) * numMatrices + tid;
        d[ tid + i * numMatrices ] = inputMatrices[ base_last ];
        if (Ncols > Nrows) {{
            unsigned int base_last = (i * Ncols + (i + 1)) * numMatrices + tid;
            e[ tid + i * numMatrices ] = inputMatrices[ base_last ];
        }}
    }}
    }}  // extern "C"
    """

Tridiagonal matrices computations

In [ ]:
def makeTridiagonalMatricesComputationKernel(dtype_str, numMatrices, Nrows, Ncols, LEN):
    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}
    #define Ncols {Ncols}
    #define LEN {LEN}

    extern "C" {{

      __global__ void tridiagFromBidiagKernel(
          const T * __restrict__ d,    // [numMatrices*Nrows] interlacciato
          const T * __restrict__ e,    // [numMatrices*LEN_BETA] interlacciato
                T * __restrict__ alpha,// [numMatrices*Ncols] interlacciato
                T * __restrict__ beta) // [numMatrices*(Ncols-1)] interlacciato
      {{
          const unsigned int tid = blockIdx.x * blockDim.x + threadIdx.x;
          if (tid >= numMatrices) return;

          alpha[0 * numMatrices + tid] = d[0 * numMatrices + tid] * d[0 * numMatrices + tid];
          beta [0 * numMatrices + tid] = d[0 * numMatrices + tid] * e[0 * numMatrices + tid];

          // i = 1 … Nrows-1  (diagonale)
          #pragma unroll
          for (unsigned int i = 1; i < Nrows; ++i) {{
                T di = d[i * numMatrices + tid];
                T ei_1 = e[(i - 1) * numMatrices + tid];
                alpha[i * numMatrices + tid] = di * di + ei_1 * ei_1;
          }}

          // βᵢ = dᵢ * eᵢ  per i = 1 … LEN -1
          #pragma unroll
          for (unsigned int i = 1; i < LEN; ++i)
                  beta[i*numMatrices + tid] = d[i*numMatrices + tid] * e[i*numMatrices + tid];

          #pragma unroll
          for (unsigned int i = Nrows; i < Ncols; ++i) {{
          //for (unsigned int i = Nrows; i < Nrows + 1; ++i) {{
              // qui non c'è più d[i] ma solo e[i-1]
              if (i == Nrows) {{
                T ei_1 = e[(i - 1) * numMatrices + tid];
                alpha[i * numMatrices + tid] = ei_1 * ei_1;
              }}
              else
                alpha[i * numMatrices + tid] = T(0);
          }}
      }}

    }} // extern "C"
    """

Pivots computation

In [ ]:
def makePivotsComputationKernel(dtype_str, numMatrices, Nrows):
    eps_str = "FLT_EPSILON" if dtype_str=="float" else "DBL_EPSILON"
    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}

    #include <cfloat>


    // pick machineâ€“epsilon for our T
    #define eps {eps_str}

    // Macro absolute / max
    //#define fabsT(x) ((T)fabs(x))
    __device__ __forceinline__ float  fabsT(float  x) {{ return fabsf(x); }}
    __device__ __forceinline__ double fabsT(double x) {{ return fabs (x); }}
    //#define fmaxT(x,y) ((T)fmax(x,y))
    __device__ __forceinline__ float  fmaxT(float  x, float y) {{ return fmaxf(x, y); }}
    __device__ __forceinline__ double fmaxT(double x, double y) {{ return fmax (x, y); }}

    extern "C" {{

    __global__ void pivotsComputationKernel(
        const T * __restrict__ b,   // [numMatrices * LEN]
              T * __restrict__ piv) // [numMatrices]
    {{
      unsigned tid = blockIdx.x * blockDim.x + threadIdx.x;
      if (tid >= numMatrices) return;

      const unsigned LEN = Nrows - 1;
      // Trovo il massimo assoluto di b[0..LEN-1]
      T localMax = fabsT(b[0*numMatrices + tid]);
      #pragma unroll
      for (unsigned i = 1; i < LEN; ++i) {{
        T v = fabsT(b[i*numMatrices + tid]);
        if (v > localMax) localMax = fmaxT(localMax, v);
      }}

      // pivot = Î²_max * eps * safety_factor (qui scelto 10)
      piv[tid] = localMax * eps * (T)5;
    }}

    }} // extern "C"

    """

Intervals computation

In [ ]:
def makeComputeInitialIntervalsKernel(dtype_str, numMatrices, Nrows):
    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}

    // file: initial_interval_kernel.cu
    #include <cfloat>
    #include <cmath>

  __device__ __forceinline__ float  fabsT(float  x) {{ return fabsf(x); }}
  __device__ __forceinline__ double fabsT(double x) {{ return fabs (x); }}

    extern "C" {{

    // ===============================================
    // KERNEL: computeInitialIntervalKernel
    // ===============================================
    __global__ void computeInitialIntervalsKernel(
        const T * __restrict__ d,       // [numMatrices * Nrows]
        const T * __restrict__ b,       // [numMatrices * (Nrows-1)]  (se Nrows<=Ncols)
    // output:
        T * __restrict__ alpha_out,     // [numMatrices]
        T * __restrict__ beta_out)      // [numMatrices]
    {{
        unsigned tid = blockIdx.x * blockDim.x + threadIdx.x;
        if (tid >= numMatrices) return;

        // 1) prendo d[0], b[0]
        T a = d[0*numMatrices + tid] - fabsT(b[0*numMatrices + tid]);
        T be = fabsT(d[0*numMatrices + tid]) + fabsT(b[0*numMatrices + tid]);

        // 2) ultimo elemento (i=Nrows-1) usa solo d[Nrows-1] e b[Nrows-2]
        {{
            unsigned i = Nrows-1;
            T di   = d[i*numMatrices + tid];
            T bi_1 = b[(i-1)*numMatrices + tid];
            T t1   = di - fabsT(bi_1);
            T t2   = fabsT(di) + fabsT(bi_1);
            if (t1 < a)     a = t1;
            if (t2 > be)    be = t2;
        }}

        // 3) generica i=1..Nrows-2
        #pragma unroll
        for (unsigned i = 1; i < Nrows-1; ++i) {{
            T di    = d[i*numMatrices + tid];
            T bi_1  = b[(i-1)*numMatrices + tid];
            T bi    = b[i*numMatrices + tid];
            T t1 = di - fabsT(bi_1) - fabsT(bi);
            T t2 = di + fabsT(bi_1) + fabsT(bi);
            if (t1 < a)  a = t1;
            if (t2 > be) be = t2;
        }}

        alpha_out[tid] = a;
        beta_out [tid] = be;
    }}

    }} // extern "C"
    """

Roots computation

In [ ]:
def makeRootsComputationKernel(dtype_str, numMatrices, Nrows, Ncols):
    return f"""
    #define T {dtype_str}
    #define numMatrices {numMatrices}
    #define Nrows {Nrows}
    #define Ncols {Ncols}

  __device__ __forceinline__ float  fabsT(float  x) {{ return fabsf(x); }}
  __device__ __forceinline__ double fabsT(double x) {{ return fabs (x); }}

  extern "C" {{
  __global__ void sturmBisectionKernel(
      const T* __restrict__ dd,           // diagonale tridiagonale interlacciata [numMatrices*Ncols]
      const T* __restrict__ bb,           // off‑diagonale tridiagonale interlacciata [numMatrices*(Ncols‑1)]
      const T* __restrict__ pivots,       // pivot per ciascuna matrice [numMatrices]
      const T* __restrict__ alpha,        // estremi inferiori [numMatrices]
      const T* __restrict__ beta,         // estremi superiori [numMatrices]
      T*       __restrict__ singularVals, // output valori singolari [numMatrices*Nrows]
      const T  tol)                       // tolleranza bisezione
  {{
      int tidx = blockIdx.x * blockDim.x + threadIdx.x;  // indice di matrice
      int tidy = blockIdx.y * blockDim.y + threadIdx.y;                            // indice di autovalore

      if (tidx >= numMatrices || tidy >= Nrows) return;

      // 1) recupero intervallo e pivot
      T a     = alpha[tidx];
      T b     = beta [tidx];
      T piv   = pivots[tidx];
      T c     = (T)0;
      unsigned numChanges;

      // 2) bisezione
      T dist = fabsT(b - a);
      T s    = fabsT(b) + fabsT(a);
      while (dist > tol * s) {{
          c = (a + b) * (T)0.5;
          // conto il numero di cambi di segno della sequenza di Sturm
          numChanges = 0;
          // primo termine
          T q = dd[   tidx] - c;
          if (fabsT(q) <= piv) q = -piv;
          if (q < (T)0) ++numChanges;
          // ricorriamo alla formula ricorsiva per i=2..Ncols
          #pragma unroll
          for (unsigned i = 2; i <= Ncols; ++i) {{
              T di   = dd[   (i-1)*numMatrices + tidx];
              T bi_1 = bb[(i-2)*numMatrices + tidx];
              q = (di - (bi_1*bi_1)/q) - c;
              if (fabsT(q) <= piv) q = -piv;
              if (q < (T)0) ++numChanges;
          }}
          // se i numeri di radici ≤ c superano Ncols - (eigIndex+1), restringo a [a,c], altrimenti [c,b]
          if (numChanges > (Ncols - (tidy + 1))) b = c;
          else                                  a = c;

          dist = fabsT(b - a);
          s    = fabsT(b) + fabsT(a);
      }}

      // 3) scrivo la radice (= valore singolare) nell’output
      singularVals[ tidy + tidx * Nrows ] = sqrt( fabsT(c) );
  }}
  }} // extern "C"
    """

Compilation

In [ ]:
rearrangeKernel                 = SourceModule(makeRearrangeKernel(dtype_str, numMatrices, Nrows, Ncols), options=["--std=c++11"], no_extern_c=True)
bidiagonalizeKernel             = SourceModule(makeBidiagonalizeKernel(dtype_str, numMatrices, Nrows, Ncols), options=["--std=c++11"], no_extern_c=True)
extractDiagonalsKernel          = SourceModule(makeExtractDiagonalsKernel(dtype_str, numMatrices, Nrows, Ncols), options=["--std=c++11"], no_extern_c=True)
tridiagonalizeKernel            = SourceModule(makeTridiagonalMatricesComputationKernel(dtype_str, numMatrices, Nrows, Ncols, LEN), options=["--std=c++11"], no_extern_c=True)
pivotsComputationKernel         = SourceModule(makePivotsComputationKernel(dtype_str, numMatrices, Nrows), options=["--std=c++11"], no_extern_c=True)
computeInitialIntervalsKernel   = SourceModule(makeComputeInitialIntervalsKernel(dtype_str, numMatrices, Nrows), options=["--std=c++11"], no_extern_c=True)
rootsComputationKernel          = SourceModule(makeRootsComputationKernel(dtype_str, numMatrices, Nrows, Ncols), options=["--std=c++11"], no_extern_c=True)

Recover the function pointers

In [ ]:
rearrange               = rearrangeKernel.get_function("rearrangeKernel")
bidiagonalize           = bidiagonalizeKernel.get_function("bidiagonalizeKernel")
extractDiagonals        = extractDiagonalsKernel.get_function("extractDiagonalsKernel")
tridiagonalize          = tridiagonalizeKernel.get_function("tridiagFromBidiagKernel")
pivotsComputation       = pivotsComputationKernel.get_function("pivotsComputationKernel")
intervalsComputation    = computeInitialIntervalsKernel.get_function("computeInitialIntervalsKernel")
rootsComputation        = rootsComputationKernel.get_function("sturmBisectionKernel")

In [ ]:
def computeBatchedSVs(input_gpu_f, output_gpu_f, d_d, d_e, d_dd, d_bb, d_piv, d_alpha, d_beta, d_sv, numMatrices, Nrows, Ncols, workType):

  bytes_per_elem      = np.dtype(workType).itemsize

  blockSize           = 32
  gridSize            = (int(numMatrices) + blockSize - 1) // blockSize  # = 1

  #########################
  # MATRIX REARRANGEMENTS #
  #########################
  rearrange(input_gpu_f, output_gpu_f, block=(blockSize,1,1), grid=(gridSize,1))
  drv.Context.synchronize()
  #print('rearrange done')

  # --- Check tridiagonal extraction
  '''h_output_gpu_f = np.zeros(numMatrices * Ncols * Nrows,     dtype=workType)
  drv.memcpy_dtoh(h_output_gpu_f, output_gpu_f)

  # Reshape and print for each matrix in batch
  for k in range(numMatrices):
      diag   = h_output_gpu_f   [k::numMatrices]      # picks [d0,d1,...] for batch k
      print(f"Rearrange batch {k}: matrix = {diag}")'''

  #####################
  # BIDIAGONALIZATION #
  #####################
  bidiagonalize(output_gpu_f, maxIter, block=(blockSize,1,1), grid=(gridSize,1) )
  drv.Context.synchronize()
  #print('bidiagonalize done')

  #####################
  # EXTRACT DIAGONALS #
  #####################
  extractDiagonals(output_gpu_f, d_d, d_e, block=(blockSize,1,1), grid=(gridSize,1,1))
  drv.Context.synchronize()
  #print('extractDiagonals done')

  ######################################################
  # TRIDIAGONAL MATRICES COMPUTATIONS FROM BIDIAGONALS #
  ######################################################
  tridiagonalize(d_d, d_e, d_dd, d_bb, block=(blockSize,1,1), grid=(gridSize,1) )
  drv.Context.synchronize()
  #print('tridiagonalize done')

  # --- Check tridiagonal extraction
  '''h_dd = np.zeros(numMatrices * Ncols,     dtype=workType)
  h_bb = np.zeros(numMatrices * (Ncols - 1), dtype=workType)
  drv.memcpy_dtoh(h_dd, d_dd)
  drv.memcpy_dtoh(h_bb, d_bb)

  # Reshape and print for each matrix in batch
  for k in range(numMatrices):
      diag   = h_dd   [k::numMatrices]      # picks [d0,d1,...] for batch k
      offdiag= h_bb   [k::numMatrices]      # picks [e0,e1,...] for batch k
      print(f"Batch {k}: dd = {diag}, ee = {offdiag}")'''


  ######################
  # PIVOTS COMPUTATION #
  ######################
  pivotsComputation(d_bb, d_piv, block=(blockSize,1,1), grid=(gridSize,1) )
  drv.Context.synchronize()
  #print('pivotsComputation done')

  #####################
  # COMPUTE INTERVALS #
  #####################
  intervalsComputation(d_dd, d_bb, d_alpha, d_beta, block=(blockSize,1,1), grid=(gridSize,1))
  drv.Context.synchronize()
  #print('intervalsComputation done')

  # --- Check interval computation
  '''h_alpha = np.zeros(numMatrices,     dtype=workType)
  h_beta = np.zeros(numMatrices, dtype=workType)
  drv.memcpy_dtoh(h_alpha, d_alpha)
  drv.memcpy_dtoh(h_beta, d_beta)

  # Reshape and print for each matrix in batch
  for k in range(numMatrices):
      diag   = h_alpha   [k::numMatrices]      # picks [d0,d1,...] for batch k
      offdiag= h_beta   [k::numMatrices]      # picks [e0,e1,...] for batch k
      print(f"Compute intervals batch {k}: alpha = {diag}, beta = {offdiag}")'''

  #####################
  # ROOTS COMPUTATION #
  #####################

  bx = 16
  by = 16
  gx = (int(numMatrices) + bx - 1) // bx
  gy = (int(Nrows) + by - 1) // by

  rootsComputation(d_dd, d_bb, d_piv, d_alpha, d_beta, d_sv, sturmTol, block=(bx, by, 1), grid=(gx, gy, 1))
  drv.Context.synchronize()
  #print('rootsComputation done')

Standard random matrix generation

In [ ]:
h_matrices = np.asfortranarray(
    np.random.randn(numMatrices, Nrows, Ncols).astype(workType)
)

'h_matrices = np.asfortranarray(\n    np.random.randn(numMatrices, Nrows, Ncols).astype(workType)\n)'

Random matrix generation according to a prescribed pattern of the singular values.

In [ ]:
'''m = Nrows = Ncols

base_kappas = np.array([2,4,6,8,10])
# e.g. repeat each value 10 times
kappas = np.repeat(base_kappas, numMatrices // len(base_kappas))
# if numMatrices isn't a multiple of 5, append a few more to reach it:
while kappas.size < numMatrices:
    kappas = np.concatenate([kappas, base_kappas[: (numMatrices - kappas.size)]])
# preallocate
batch = np.empty((numMatrices, m, m), dtype=workType, order='F')

for j, κ in enumerate(kappas):
    # 1) geometric spectrum from cond=1 to cond=10^κ
    s = 10**np.linspace(0, κ, m)

    # 2) random orthonormal U, V via QR
    #    (np.linalg.qr returns Q,R; we only need Q)
    U, _ = np.linalg.qr(np.random.randn(m, m))
    V, _ = np.linalg.qr(np.random.randn(m, m))

    # 3) assemble
    A = U @ np.diag(s) @ V.T

    # store (astype and Fortran‐order)
    batch[j, :, :] = A.astype(workType, copy=False)

# ensure entire batch is Fortran‐contiguous
h_matrices = np.asfortranarray(batch)'''

"m = Nrows = Ncols\n\nbase_kappas = np.array([2,4,6,8,10])\n# e.g. repeat each value 10 times\nkappas = np.repeat(base_kappas, numMatrices // len(base_kappas))\n# if numMatrices isn't a multiple of 5, append a few more to reach it:\nwhile kappas.size < numMatrices:\n    kappas = np.concatenate([kappas, base_kappas[: (numMatrices - kappas.size)]])\n# preallocate\nbatch = np.empty((numMatrices, m, m), dtype=workType, order='F')\n\nfor j, κ in enumerate(kappas):\n    # 1) geometric spectrum from cond=1 to cond=10^κ\n    s = 10**np.linspace(0, κ, m)\n\n    # 2) random orthonormal U, V via QR\n    #    (np.linalg.qr returns Q,R; we only need Q)\n    U, _ = np.linalg.qr(np.random.randn(m, m))\n    V, _ = np.linalg.qr(np.random.randn(m, m))\n\n    # 3) assemble\n    A = U @ np.diag(s) @ V.T\n\n    # store (astype and Fortran‐order)\n    batch[j, :, :] = A.astype(workType, copy=False)\n\n# ensure entire batch is Fortran‐contiguous\nh_matrices = np.asfortranarray(batch)"

Random matrix generation by perturbing a fixed random matrix

In [ ]:
'''from numpy.linalg import norm

# Parameters
m = Nrows = Ncols            # assume square

epsilons = np.array([1e-8, 1e-7, 1e-6, 1e-5], dtype=workType)

# 1) Build your “clean” base matrix A
#    For example: a random Gaussian, or reuse one from your Experiment 2
A = np.random.randn(m, m).astype(workType, copy=False)

# Precompute its 2‐norm
A_norm2 = norm(A, 2)

# 2) Allocate batch container [J × m × m] in Fortran order
J = epsilons.size
batch = np.empty((J, m, m), dtype=workType, order='F')

# 3) Fill in each noisy realization
for j, eps in enumerate(epsilons):
    # draw noise matrix G ~ N(0,1)
    G = np.random.randn(m, m).astype(workType, copy=False)
    # form noisy matrix
    batch[j, :, :] = (A + eps * A_norm2 * G)

# 4) Ensure Fortran‐contiguity (batch was created as such)
h_matrices = np.asfortranarray(batch)'''

In [ ]:
# Concatenate the random matrices
h_input_col = np.zeros(numMatrices * Nrows * Ncols, dtype=workType)
for k in range(numMatrices):
    h_input_col[k*(Nrows*Ncols):(k+1)*(Nrows*Ncols)] = h_matrices[k].flatten(order='F')

input_gpu_f       = drv.mem_alloc(h_input_col.nbytes)
drv.memcpy_htod(input_gpu_f, h_input_col)

bytes_per_elem      = np.dtype(workType).itemsize
d_sv                = drv.mem_alloc(int(numMatrices * Nrows * bytes_per_elem))

h_sv                = np.zeros(numMatrices * Nrows, dtype=workType);

output_gpu_f        = drv.mem_alloc(int(numMatrices * Ncols * Nrows * bytes_per_elem))

# Allocate GPU space for diagonals
size_d              = numMatrices * Nrows * bytes_per_elem
size_e              = numMatrices * LEN   * bytes_per_elem
d_d                 = drv.mem_alloc(int(size_d))                                  # --- Main diagonal of the bidiagonal matrix
d_e                 = drv.mem_alloc(int(size_e))                                  # --- Upper diagonal of the bidiagonal matrix

size_dd             = numMatrices * Ncols * bytes_per_elem
size_bb             = numMatrices * (Ncols - 1) * bytes_per_elem
d_dd                = drv.mem_alloc(int(size_dd))                                 # --- alpha tridiagonal
d_bb                = drv.mem_alloc(int(size_bb))                                 # --- beta tridiagonal

size_piv            = numMatrices * bytes_per_elem
d_piv               = drv.mem_alloc(int(size_piv))

size_alpha_beta     = numMatrices * bytes_per_elem
d_alpha             = drv.mem_alloc(int(size_alpha_beta))
d_beta              = drv.mem_alloc(int(size_alpha_beta))

Warmup execution

In [ ]:
computeBatchedSVs(input_gpu_f, output_gpu_f, d_d, d_e, d_dd, d_bb, d_piv, d_alpha, d_beta, d_sv, numMatrices, Nrows, Ncols, workType)
drv.memcpy_dtoh(h_sv, d_sv)
sv_matrix         = h_sv.reshape((numMatrices, Nrows), order='C')

err_avg = 0.
for k in range(numMatrices):
    A_k = h_matrices[k]
    s = np.linalg.svd(A_k, compute_uv=False)
    # Compare only the first min(Nrows, Ncols) singular values
    err = 100. * np.linalg.norm(s - sv_matrix[k][:min(Nrows, Ncols)]) / np.linalg.norm(s)
    print(err)
    #print(s)
    err_avg += err

err_avg = err_avg / numMatrices
    #print(f"Matrix {k}:  numpy SVD singular values = {s}")
    #print(f"Matrix {k}: ", sv_matrix[k])

print(f"Batch size numMatrices = {numMatrices} → avg error: {err_avg:.3e}")

2.4258969916499614e-12
3.3036979039454832e-12
3.4929352991204703e-12
2.7666183258485706e-12
Batch size numMatrices = 4 → avg error: 2.997e-12


True execution

In [ ]:
# Prepare CUDA events for timing
start = drv.Event()
end   = drv.Event()
# Record start
start.record()
computeBatchedSVs(input_gpu_f, output_gpu_f, d_d, d_e, d_dd, d_bb, d_alpha, d_beta, d_piv, d_sv, numMatrices, Nrows, Ncols, workType)
end.record()
end.synchronize()
elapsed_ms = start.time_till(end)

#h_sv              = np.zeros(numMatrices * Nrows, dtype=workType)
drv.memcpy_dtoh(h_sv, d_sv)
sv_matrix         = h_sv.reshape((numMatrices, Nrows), order='C')

#print("\n=== Comparison with NumPy SVD ===")
err_avg = 0.
for k in range(numMatrices):
    A_k = h_matrices[k]
    s = np.linalg.svd(A_k, compute_uv=False)
    #err = 100. * np.linalg.norm(s - sv_matrix[k]) / np.linalg.norm(s)
    err_avg += 100. * np.linalg.norm(s - sv_matrix[k][:min(Nrows, Ncols)]) / np.linalg.norm(s)
    #print(f"Matrix {k}: ", sv_matrix[k])
    #print(f"Matrix {k}:  numpy SVD singular values = {s}")

err_avg = err_avg / numMatrices

print(f"Batch size numMatrices = {numMatrices} → elapsed: {elapsed_ms / 1000:.6f} s, avg error: {err_avg:.3e}")

Batch size numMatrices = 4 → elapsed: 0.000624 s, avg error: 2.997e-12
